# Experiment 1

### Environment Setup

In [1]:
%%bash -s

# set root folder and get environment
export root="/home/fabric/work"
source $root/env
pwd

/home/fabric/work


In [2]:
# imports
import os
import time
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from fabrictestbed_extensions.fablib.node import Node
from threading import Thread
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import uuid
from collections import defaultdict
from IPython.display import display

# show fablib config
fablib = fablib_manager()             
fablib.show_config()

# constants
root = "/home/fabric/work"

# component names
slice_name = "experiment1"
server_site, client_site = fablib.get_random_sites(count=2, avoid=['AL2S','STAR', 'MAX','TACC','NCSA'])
print(f"{server_site=}, {client_site=}")

# switch detail
client_switch_name = "client-switch"
server_switch_name = "server-switch"
switch_cores = 2
switch_ram = 8
switch_disk = 100

# host details
client_host_name = "client-host"
server_host_name = "server-host"
image = 'default_ubuntu_20'
server_host_subnet=IPv4Network('10.0.1.0/24')
server_host_addr=IPv4Address('10.0.1.1')
client_host_subnet=IPv4Network('10.0.2.0/24')
client_host_addr=IPv4Address('10.0.2.2')

# network details
net_server_switch_name = "net-server-switch"
net_client_switch_name = "net-client-switch"
net_switch_switch_name = "net-switch-switch"

credmgr_host,cm.fabric-testbed.net
orchestrator_host,orchestrator.fabric-testbed.net
fabric_token,/home/fabric/.tokens.json
project_id,6ce270de-788d-4e07-8bae-3206860a6387
bastion_username,brmclemo_0000026977
bastion_key_filename,/home/fabric/work/ssh/fabric-bastion
bastion_public_addr,bastion-1.fabric-testbed.net
bastion_passphrase,None
slice_public_key_file,/home/fabric/work/ssh/fabric-sliver.pub
slice_private_key_file,/home/fabric/work/ssh/fabric-sliver
fabric_slice_private_key_passphrase,None


server_site='UTAH', client_site='CLEM'


In [3]:
# some helper functions
__id_counter = defaultdict(int)
def id(name=None):
  global __id_counter
  __id_counter[name] += 1
  return f"{name}-{str(__id_counter[name]).zfill(4)}"

## Setup Network

In [4]:

# raise

# create the slice
slice = fablib.new_slice(name=slice_name)

# add server switch
server_switch = slice.add_node(name=server_switch_name, site=server_site, image=image)
server_switch_iface_host = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
server_switch_iface_client_switch = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client switch
client_switch = slice.add_node(name=client_switch_name, site=client_site, image=image)
client_switch_iface_host = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
client_switch_iface_server_switch = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add server host
server_host = slice.add_node(name=server_host_name, site=server_site, image=image)
server_host_iface = server_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client host
client_host = slice.add_node(name=client_host_name, site=client_site, image=image)
client_host_iface = client_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add networks
server_switch_net = slice.add_l2network(name=net_server_switch_name, interfaces=[server_switch_iface_host, server_host_iface])
client_switch_net = slice.add_l2network(name=net_client_switch_name, interfaces=[client_switch_iface_host, client_host_iface])
switch_switch_net = slice.add_l2network(name=net_switch_switch_name, interfaces=[server_switch_iface_client_switch, client_switch_iface_server_switch])

# submit slice for creation
slice.submit()


RuntimeError: No active exception to reraise

In [ ]:
# set up nodes
ip_addrs = dict()
slice = fablib.get_slice(name=slice_name)
available_addrs = list(subnet := IPv4Network("192.168.2.0/24"))[1:]
gateway_addr = available_addrs.pop(0)
addrs = defaultdict(lambda: available_addrs.pop(0))
for node in slice.get_nodes():
  node_name = node.get_name()
  print(f"{node_name=}")

  # setup
  node.execute("sudo apt update && sudo apt install net-tools && pip install netifaces")
  node.upload_directory('code','~')

  # give ip to all nodes
  for iface in node.get_interfaces():
    iface_name = iface.get_name()
    print(f"{iface_name=}")
    if 'host' in node_name:
      iface.ip_addr_add(addr=addrs[node_name], subnet=subnet)
    else:
      iface.ip_addr_add(addr=gateway_addr, subnet=subnet)
    iface.ip_link_up()
    
  


In [ ]:
# set up l3 networking addresses
l3_available_addrs = list(l3_subnet := IPv4Network("192.168.3.0/24"))[1:]
l3_addrs = defaultdict(lambda: l3_available_addrs.pop(0))
for node in slice.get_nodes():
  node_name = node.get_name()
  if 'host' in node_name:
    print(f"{node_name=}")
    for iface in node.get_interfaces():
      iface_name = iface.get_name()
      print(f"{iface_name=}")
      iface.ip_addr_add(addr=l3_addrs[node_name], subnet=l3_subnet)


# static arp tables
for node in slice.get_nodes():
  node_name = node.get_name()
  if 'host' in node_name:
    print(f"{node_name=}")
    # "sudo ip neighbor add 192.168.3.8 dev ens7 lladdr ff:ff:ff:ff:ff:ff"
    command = '; '.join(
      f"sudo ip neighbor add {l3_addr} dev {iface.get_os_interface()} lladdr ff:ff:ff:ff:ff:ff"
      for iface in node.get_interfaces()
      for l3_addr in l3_addrs.values()
    )
    node.execute(command)
    

In [ ]:
# store values for server
for node in slice.get_nodes():
  node_name = node.get_name()
  if 'switch' in node_name:
    print(f"{node_name=}")
    host_iface = node.get_interface(network_name=f"net-{node_name}")
    network_iface = node.get_interface(network_name=f"net-switch-switch")
    transformer_config = {
      'host': host_iface.get_os_interface(),
      'network': network_iface.get_os_interface(),
      'id': 0,
      'protocols': {
        # 'ARP': 0x0806,
        'IP': 0x0800,
      }
    }
    transformer_config_json = json.dumps(transformer_config)
    node.execute(f"echo '{transformer_config_json}' >transformer_config.json")
    node.execute(f"echo identity >identity.txt")
# for net in slice.get_networks():
#   net_name=net.get_name()
#   print(f"{net.get_name()=}")

In [ ]:
# keep track of threads
threads = {}

In [ ]:
# start servers
raise
timeout = 10
slice.get_node(server_switch_name).execute_thread(f"sudo timeout {timeout} python3 code/transformer.py <transformer_config.json")
slice.get_node(client_switch_name).execute_thread(f"sudo timeout {timeout} python3 code/transformer.py <transformer_config.json")
slice.get_node(server_host_name).execute_thread(f"nc -ul 8080 -w {timeout} >result.txt")
time.sleep(2)
slice.get_node(client_host_name).execute_thread(f"nc -u {l3_addrs[server_host_name]} -w {timeout} <identity.txt")
time.sleep(10)
print('here')
out, err = slice.get_node(server_host_name).execute("cat result.txt")
print(out)

### Configure Hosts

### Configure Switches

## Delete Slice